At this point there are multiple agents. But they act indepdently. There is no distribution/exchange mechanism for the agents.

Also, the product sector chosen by the agents is randomly selected after they choose to make the production decision. So division of labor will not be correct.

Include each product sector as a unique action. Should also include sector productivity input for agent's Deep Q Network

When agents can possess multiple instruments, will need to generalize or revisit the above.

In [14]:
import Agents
import Instruments
import Simulate
import Environment
import SimPlotting

import numpy as np
from scipy.stats import norm, uniform
import random

import time

import matplotlib.pyplot as plt

In [15]:
def initialize_agents(Nagents, initial_parameters, learning_parameters) -> list:

    kwargs = {**initial_parameters, **learning_parameters}
    
    
    Agent_list = [Agents.Agent(**kwargs) for i in range(Nagents)]

    return Agent_list

def initialize_instruments(Ninstruments, means, stds):
    '''
    Assumes Normal. Initializes Universal Instruments.
    
    Make more general to incude different distributions'''

    instruments = []

    for N in range(Ninstruments):
        prod_vec = [norm.rvs(loc=mean, scale=std) for mean, std in zip(means, stds)]
        instrument = Instruments.Universal(prod_vec)
        instruments.append(instrument)
        
    return instruments

def initialize_allocation(agent_list, instrument_list):
    
    for agent in agent_list:
        
        random_instrument = random.choice(instrument_list)
        instrument_list.remove(random_instrument)
        
        agent.Ins = random_instrument
        
def main(Ntimes, Nagents):
    
    import Agents
    import Instruments
    import Simulate
    import Environment
    import SimPlotting

    import numpy as np
    from scipy.stats import norm, uniform
    import random

    import time

    import matplotlib.pyplot as plt

    dt = 1

    cg = np.array([1.0, 1.0])

    Nproducts = cg.shape[0]

    Nactions = 3 + Nproducts # One production decision for each product + null + exchange + consume

    mean_productivities = [5, 5]
    std_productivities = [0, 0]

    Ninstruments = Nagents

    initial_parameters = {
        'Q': np.zeros(Nproducts),
        'D': np.zeros(Nproducts),
        'cg': cg,
         'M': 10.,
    }

    learning_parameters = {
        'gamma': 0.9,
        #'gamma': 0.5,
        #'gamma': 0.1,
        'epsilon': 1.0,
        'eps_end': 0.001,
        'eps_dec': 0.01,
        #'batch_size': 100,
        'batch_size': 10,
        'input_dims': [3*Nproducts+1],
        'lr':0.001,
        #'lr': 0.01,
        'n_actions': Nactions,
    }

    agent_list = initialize_agents(Nagents, initial_parameters, learning_parameters)
    instrument_list = initialize_instruments(Ninstruments, mean_productivities, std_productivities)
    market_list = [0]

    initialize_allocation(agent_list, instrument_list)

    env = Environment.Environment(dt, agent_list, instrument_list, market_list)
    sim = Simulate.Simulate(env)

    #start = time.perf_counter()
    sim.run_simulation(Ntimes)
    #end = time.perf_counter()

    #elapsed_time = end-start

    #return elapsed_time

In [16]:
import cProfile

In [17]:
main(10,1)

In [18]:
cProfile.run('main(200,10)', 'restats')

In [19]:
import pstats
from pstats import SortKey

In [20]:
p = pstats.Stats('restats')

 The sort_stats() method sorted all the entries according to the standard module/line/name string that is printed. The print_stats() method printed out all the statistics

In [21]:
#p = pstats.Stats('restats')
#p.strip_dirs().sort_stats(-1).print_stats()

The first call will actually sort the list by function name, and the second call will print out the statistics.

In [22]:
#p.sort_stats(SortKey.NAME)
#p.print_stats()

This sorts the profile by cumulative time in a function.

In [23]:
p.sort_stats(SortKey.CUMULATIVE).print_stats(30)

Fri Jul 21 01:13:42 2023    restats

         744238 function calls (743968 primitive calls) in 13.741 seconds

   Ordered by: cumulative time
   List reduced from 228 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   13.741   13.741 {built-in method builtins.exec}
        1    0.004    0.004   13.741   13.741 <string>:1(<module>)
        1    0.000    0.000   13.737   13.737 C:\Users\Alex\AppData\Local\Temp\ipykernel_21232\718577105.py:34(main)
        1    0.002    0.002   13.709   13.709 C:\Users\Alex\Research\EconoNet\2023-07-18_MultipleSectors\Simulate.py:138(run_simulation)
      200    0.245    0.001   13.708    0.069 C:\Users\Alex\Research\EconoNet\2023-07-18_MultipleSectors\Simulate.py:156(sim_step)
     2000    0.719    0.000   12.285    0.006 C:\Users\Alex\Research\EconoNet\2023-07-18_MultipleSectors\Agents.py:159(learn)
     1910    0.097    0.000    4.424    0.002 C:\Users\Alex\anaconda3\env

If you were looking to see what functions were looping a lot, and taking a lot of time.

Sort according to time spent within each function, and then print the statistics for the top $x$ functions.

In [24]:
p.sort_stats(SortKey.TIME).print_stats(30)

Fri Jul 21 01:13:42 2023    restats

         744238 function calls (743968 primitive calls) in 13.741 seconds

   Ordered by: internal time
   List reduced from 228 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1910    3.845    0.002    3.845    0.002 {method 'run_backward' of 'torch._C._EngineBase' objects}
    15639    1.732    0.000    1.732    0.000 {built-in method torch._C._nn.linear}
     1910    0.942    0.000    3.707    0.002 C:\Users\Alex\anaconda3\envs\pytorchenv\lib\site-packages\torch\optim\adam.py:300(_single_tensor_adam)
     2000    0.719    0.000   12.285    0.006 C:\Users\Alex\Research\EconoNet\2023-07-18_MultipleSectors\Agents.py:159(learn)
    11460    0.693    0.000    0.693    0.000 {method 'sqrt' of 'torch._C._TensorBase' objects}
    22920    0.608    0.000    0.608    0.000 {method 'mul_' of 'torch._C._TensorBase' objects}
    11460    0.472    0.000    0.472    0.000 {method 'addcdiv_' of 'torch._

In [25]:
#p.sort_stats(SortKey.FILENAME).print_stats('__init__')

In [26]:
#p.sort_stats(SortKey.TIME, SortKey.CUMULATIVE).print_stats(.5)